In [ ]:
# Genetic algorithms for solving Quadratic Assigment Problems

In [120]:
import numpy as np

# Defining the problem
class QAProblem:

    def __init__(self, n, distances, flows):
        self.n = n
        self.distances = distances
        self.flows = flows

    def trad_eval(self, solution):
        cost = 0.
        for i in range(self.n):
            for j in range(self.n):
                dist = self.distances[solution[i]][solution[j]]
                flow = self.flows[i][j]
                cost += flow * dist
        return cost

    def symm_eval(self, solution):
        cost = 0
        for i in range(self.n-1):
            for j in range(i+1, self.n):
                dist = self.distances[solution[i]][solution[j]]
                flow = self.flows[i][j]
                cost += 2*(flow * dist)
        return cost

    def __call__(self, solution):
        perm_dist = self.distances[solution,:][:,solution]
        return int(np.multiply(perm_dist, self.flows).sum())

    def create_from_file(path):
        with open(path, "r") as f:
            n = int(f.readline().strip())
            distances, flows = np.zeros((n, n), dtype=int), np.zeros((n, n), dtype=int)
            _ = f.readline()
            for i in range(n):
                flows[i,:] = (list(map(int, f.readline().split())))
            for j in range(n):
                distances[j,:] = (list(map(int, f.readline().split())))
        return QAProblem(n, distances, flows)

# Reading data

qaProblem = QAProblem.create_from_file(path="qap_data/tai256c.dat")

print(np.shape(qaProblem.distances))
print(np.shape(qaProblem.flows))


(256, 256)
(256, 256)


In [121]:
# Testing evaluation approaches
import time

np.random.seed(123)


# Number of solutions (individuals)
pop_size = 50

# Creating the solution set, (population)
pop = [np.random.permutation(qaProblem.n) for i in range(pop_size)]

# Collecting fitness evaluations
pop_fits = np.zeros((pop_size, 3), dtype=int)


# Traditional approache
start = time.time()
fits = []
for ind in pop:
    fits.append(qaProblem.trad_eval(ind))
pop_fits[:, 0] = fits
end = time.time()
print(end - start)

# Exploiting symmetry
start = time.time()
fits = []
for ind in pop:
    fits.append(qaProblem.symm_eval(ind))
pop_fits[:, 1] = fits
end = time.time()
print(end - start)

# Exploiting numpy multiplication and sum
start = time.time()
fits = []
for ind in pop:
    fits.append(qaProblem(ind))
pop_fits[:, 2] = fits
end = time.time()
print(end - start)

# Checking fitness equallity between approaches

print((pop_fits[:,0] == pop_fits[:,1]).all())
print((pop_fits[:,0] == pop_fits[:,2]).all())
print((pop_fits[:,1] == pop_fits[:,2]).all())




2.912590980529785
1.5737297534942627
0.01387786865234375
True
True
True


In [123]:
class GAforQA:
    def __init__(self, qa_problem:QAProblem, pop_size:int=50, crossover_rate:float=0.7, mutation_rate:float=0.1) -> None:
        self.qa_problem = qa_problem
        self.pop_size = pop_size
        self.cross_rate = crossover_rate
        self.mutation_rate = mutation_rate
        self.population = np.empty((self.pop_size, self.qa_problem.n),  dtype=int)
        self.fitness = np.zeros(self.pop_size, dtype=int)

    def initialize(self):
        for i in range(self.pop_size):
            x = np.random.permutation(self.qa_problem.n)
            f = self.qa_problem(x)
            self.population[i,:] = x
            self.fitness[i] = f

    
    

ga_alg = GAforQA(qaProblem, pop_size=10)

ga_alg.initialize()





In [124]:
ga_alg.population
sortindx = np.argsort(ga_alg.fitness)


print(ga_alg.fitness)
print(sortindx)
print(ga_alg.fitness[sortindx])



[55603024 55026808 54207698 53111872 54152948 54031734 54390170 54114796
 55710228 52386630]
[9 3 5 7 4 2 6 1 0 8]
[52386630 53111872 54031734 54114796 54152948 54207698 54390170 55026808
 55603024 55710228]
